In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus

In [69]:
op= Optimus()

In [70]:
df = op.load.url("https://raw.githubusercontent.com/ironmussa/Optimus/master/examples/foo.csv")

In [12]:
df.table()

id 1 (int),firstName 2 (string),lastName 3 (string),billingId 4 (int),product 5 (string),price 6 (int),birth 7 (string),dummyCol 8 (string)
1,Luis,Alvarez$$%!,123,Cake,10,1980/07/07,never
2,André,Ampère,423,piza,8,1950/07/08,gonna
3,NiELS,Böhr//((%%,551,pizza,8,1990/07/09,give
4,PAUL,dirac$,521,pizza,8,1954/07/10,you
5,Albert,Einstein,634,pizza,8,1990/07/11,up
6,Galileo,⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱⸱GALiLEI,672,arepa,5,1930/08/12,never
7,CaRL,Ga%%%uss,323,taco,3,1970/07/13,gonna
8,David,H$$$ilbert,624,taaaccoo,3,1950/07/14,let
9,Johannes,KEPLER,735,taco,3,1920/04/22,you
10,JaMES,M$$ax%%well,875,taco,3,1923/03/12,down


In [2]:
import redis
r = redis.StrictRedis(host='localhost', port=6379, db=0)
r.set('foo', 'bar')

ConnectionError: Error 10061 connecting to localhost:6379. No connection could be made because the target machine actively refused it.

In [ ]:
r.get('foo')